In [2]:
# Import necessary libraries
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

# Define constants
data_dir = "C:/Users/abhay/Downloads/asl_alphabet_train/asl_alphabet_train"
img_size = 64
num_classes = 29  # 26 letters + 'del', 'nothing', 'space'

# Load and preprocess the data
data = []
labels = []

for label in os.listdir(data_dir):
    path = os.path.join(data_dir, label)
    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        img_array = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        img_array = cv2.resize(img_array, (img_size, img_size))
        data.append([img_array, label])

# Shuffle the data
np.random.shuffle(data)

# Split data into features and labels
X = np.array([i[0] for i in data]).reshape(-1, img_size, img_size, 1)
y = to_categorical([ord(i[1]) - ord('A') if 'A' <= i[1] <= 'Z' else
                   26 if i[1] == 'del' else
                   27 if i[1] == 'nothing' else
                   28  # 'space'
                   for i in data], num_classes=num_classes)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Number of images: {len(data)}")
print(f"Shape of X: {X.shape}")
print(f"Shape of y: {y.shape}")

Number of images: 87000
Shape of X: (87000, 64, 64, 1)
Shape of y: (87000, 29)


In [3]:
# Import necessary libraries for building the CNN model
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Define the input shape
input_shape = (img_size, img_size, 1)

# Build the CNN model
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=input_shape, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))  # Use num_classes instead of 26

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()




Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 31, 31, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 14, 14, 64)        0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 12544)             0         
                                                                 
 dense (Dense)               (None, 128)             

In [4]:
from keras.preprocessing.image import ImageDataGenerator

# Augment images during training
datagen = ImageDataGenerator(rescale=1./255,
                             shear_range=0.2,
                             zoom_range=0.2,
                             horizontal_flip=True)

datagen.fit(X_train)

# Train the model
model.fit(datagen.flow(X_train, y_train, batch_size=32),
          steps_per_epoch=len(X_train) / 32,
          epochs=10,
          validation_data=(X_test, y_test))

Epoch 1/10


2175/2175 [==============================] - 80s 36ms/step - loss: 1.4739 - accuracy: 0.5487 - val_loss: 683.6526 - val_accuracy: 0.4087
Epoch 2/10
2175/2175 [==============================] - 76s 35ms/step - loss: 0.5564 - accuracy: 0.8180 - val_loss: 1422.0901 - val_accuracy: 0.3402
Epoch 3/10
2175/2175 [==============================] - 76s 35ms/step - loss: 0.3436 - accuracy: 0.8866 - val_loss: 2655.7476 - val_accuracy: 0.2802
Epoch 4/10
2175/2175 [==============================] - 76s 35ms/step - loss: 0.2453 - accuracy: 0.9197 - val_loss: 4545.0000 - val_accuracy: 0.2052
Epoch 5/10
2175/2175 [==============================] - 75s 34ms/step - loss: 0.1909 - accuracy: 0.9367 - val_loss: 6657.8994 - val_accuracy: 0.1656
Epoch 6/10
2175/2175 [==============================] - 77s 35ms/step - loss: 0.1513 - accuracy: 0.9488 - val_loss: 7839.8491 - val_accuracy: 0.1539
Epoch 7/10
2175/2175 [==============================] - 75s 35ms/step - loss: 0.1271 - accuracy: 0.9577 -

In [6]:
model.save("asl_model.keras")